<a href="https://colab.research.google.com/github/bhanuteja2001/COVID-19-XrayDetection/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/covid_image_data/filetrain (1).csv')
df.head()

,Unnamed: 0,filename,file,label
0,0,Image_ex_1.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
1,1,Image_ex_2.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
2,2,Image_ex_3.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
3,3,Image_ex_4.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
4,4,Image_ex_5.jpg,/content/drive/MyDrive/covid_image_data/train/...,0


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,filename,file,label
0,Image_ex_1.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
1,Image_ex_2.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
2,Image_ex_3.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
3,Image_ex_4.jpg,/content/drive/MyDrive/covid_image_data/train/...,0
4,Image_ex_5.jpg,/content/drive/MyDrive/covid_image_data/train/...,0


In [ ]:
df.shape

(2917, 2)

In [6]:
df.label.value_counts()

0    1779
1    1138
Name: label, dtype: int64

In [7]:
df = df.sample(frac = 1).reset_index(drop=True)

In [8]:
Train = df

In [12]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [14]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/covid_image_data/Train'
valid_path = '/content/drive/MyDrive/covid_image_data/test'


vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
  # useful for getting number of classes
folders = glob('/content/drive/MyDrive/covid_image_data/Train/*')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

58892288/58889256 [==============================] - 0s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_______________________________________________________

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/covid_image_data/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/covid_image_data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''


es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, 
                                               restore_best_weights=True,
                                               verbose=1)





# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[es_callback]
)




Found 2918 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Epoch 1/50
92/92 [==============================] - 44s 475ms/step - loss: 0.1072 - accuracy: 0.9650
Epoch 2/50
92/92 [==============================] - 44s 475ms/step - loss: 0.1042 - accuracy: 0.9650
Epoch 3/50
66/92 [====================>.........] - ETA: 12s - loss: 0.1004 - accuracy: 0.9693

KeyboardInterrupt: ignored

## Testing

In [24]:
df_testing = pd.read_csv('/content/drive/MyDrive/covid_image_data/Testing_set_covid.csv')
df_testing.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [25]:
images_testing = [[fname, '/content/drive/MyDrive/covid_image_data/test/' + fname[:-4] + '.jpg'] for fname in df_testing['filename']]
images_testing = pd.DataFrame(images_testing)
images_testing.columns = ['filename', 'file']
images_testing

,filename,file
0,Image_1.jpg,/content/drive/MyDrive/covid_image_data/test/I...
1,Image_2.jpg,/content/drive/MyDrive/covid_image_data/test/I...
2,Image_3.jpg,/content/drive/MyDrive/covid_image_data/test/I...
3,Image_4.jpg,/content/drive/MyDrive/covid_image_data/test/I...
4,Image_5.jpg,/content/drive/MyDrive/covid_image_data/test/I...
...,...,...
865,Image_866.jpg,/content/drive/MyDrive/covid_image_data/test/I...
866,Image_867.jpg,/content/drive/MyDrive/covid_image_data/test/I...
867,Image_868.jpg,/content/drive/MyDrive/covid_image_data/test/I...
868,Image_869.jpg,/content/drive/MyDrive/covid_image_data/test/I...


In [27]:
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
test_image = []
for i in tqdm(range(images_testing.shape[0])):
    img = load_img(images_testing['file'][i], target_size=(224, 224, 3), grayscale=False)
    img = img_to_array(img)
    img = img/255
    test_image.append(img)
X = np.array(test_image)

100%|██████████| 870/870 [05:04<00:00,  2.86it/s]


In [29]:
prediction = model.predict(X)

In [31]:
pd.DataFrame(prediction)[1][0]

6.776237e-05